In [ ]:
from experiments.exp_3d.preprocessing import *
from losses.Metrics import metric_dice

from lib.visualize import display_instance
from experiments.exp_3d.inference import Pipeline

import matplotlib.pyplot as plt
import pandas as pd 

import pydicom
import pydicom_seg
import SimpleITK as sitk
from library_dicom.export_segmentation.ExportSegmentation_Writer import ExportSegmentation_Writer

In [ ]:
target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)
model_path = '/media/deeplearning/LACIE SHARE/RUDY_WEIGTH/training/20201125-17_00_45/trained_model_20201125-17_00_45.h5'
target_origin = None
from_pp = False

csv_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/inference.csv'


df = pd.read_csv(csv_path)
dataset = df[['study_id', 'ct_img', 'pet_img']].to_dict('records')
print(dataset)
print("")
pipeline = Pipeline(target_size, target_spacing, target_direction, model_path=model_path, target_origin=None, from_pp=False)
print("Pipeline object initialized")

In [ ]:
for count, img_path in enumerate(dataset) : 
    study_uid = img_path['study_id']
    pet_img = sitk.ReadImage(img_path['pet_img'], sitk.sitkFloat32)
    pet_spacing = pet_img.GetSpacing()
    pet_size = pet_img.GetSize()
    pet_origin = pet_img.GetOrigin()
    pet_direction = pet_img.GetDirection()
    #pet_array = np.rot90(sitk.GetArrayFromImage(pet_img),2)
    pet_array = sitk.GetArrayFromImage(pet_img)
    print(pet_array.shape)

    axis = 1
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    fig.suptitle("id : {}".format(study_uid))
    display_instance(pet_array, mask_array=None, axis=axis, ax=ax1)
    ax1.set_title('PET scan')
    
    
    pred_img = pipeline(img_path)
    #print(str(type(pred_nifti)))
    #pred = np.rot90(sitk.GetArrayFromImage(pred_nifti),2)
    pred = sitk.GetArrayFromImage(pred_img)
    print(pred.shape)
    display_instance(pet_array, mask_array=pred, axis=axis, ax=ax2)
    ax2.set_title('prediction')

### - Generate DICOM-SEG file 

In [ ]:
segmentation = pred_img
pet_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/1.2.250.1.59.453.859.92444169.1516.1370594209.2.2.8'
filename = 'sarcome_dicom_seg.dcm'
directory_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
export_objet = ExportSegmentation_Writer(segmentation, segmentation_type='img', serie_path = pet_path, mode='dicomseg')
export_objet.generate_dicom(filename = filename, directory_path = directory_path)

### - Generate nifti 

In [ ]:
#Check here 
img = sitk.GetImageFromArray(np.flip(pred, axis = 0))
img.SetSpacing(pet_spacing)
img.SetOrigin(pet_origin)
img.SetDirection(pet_direction)

directory = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
filename = directory + '/'+ 'nifti_mask_prediction_v6.nii'
sitk.WriteImage(img, filename)

### - Generate RTSTRUCT file 

In [ ]:
from library_dicom.export_segmentation.ExportSegmentation_Writer import ExportSegmentation_Writer
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.export_segmentation.tools.rtss_writer_tools import *
from skimage.measure import label


segmentation = sitk.GetArrayFromImage(pred_img)
segmentation = clean_mask(segmentation)
segmentation = label(segmentation)

#generate rtstruct file 
pet_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome/1.2.250.1.59.453.859.92444169.1516.1370594209.2.2.8'
export_objet = ExportSegmentation_Writer(segmentation, segmentation_type ='matrix', serie_path = pet_path, mode = 'rtstruct')
filename = 'sarcome_rtstruct.dcm'
directory_path = '/media/deeplearning/LACIE SHARE/Olivier_Morel_sarcome'
export_objet.generate_dicom(filename, directory_path)